# 🫁 Pneumonia Detection Using Deep Learning | ResNet50 + TensorFlow

This notebook uses transfer learning with ResNet50 to detect Pneumonia from chest X-ray images.
Author: Ujjwal Pal

In [ ]:
# Install required libraries (if running on Colab)
# !pip install tensorflow matplotlib numpy pandas opencv-python scikit-learn

In [ ]:
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from sklearn.metrics import classification_report, confusion_matrix

## 📂 Load Dataset

In [ ]:
# Set dataset directories
base_dir = 'chest_xray'
train_dir = os.path.join(base_dir, 'train')
val_dir = os.path.join(base_dir, 'val')
test_dir = os.path.join(base_dir, 'test')

## 🌀 Data Generators

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255, zoom_range=0.2, shear_range=0.2, horizontal_flip=True)
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_dir, target_size=(224, 224), batch_size=32, class_mode='binary')
val_generator = val_datagen.flow_from_directory(val_dir, target_size=(224, 224), batch_size=32, class_mode='binary')
test_generator = test_datagen.flow_from_directory(test_dir, target_size=(224, 224), batch_size=32, class_mode='binary')

## 🧠 Build the Model using ResNet50

In [ ]:
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

## 🏋️ Train the Model

In [ ]:
history = model.fit(train_generator, validation_data=val_generator, epochs=5)

## 📊 Evaluate the Model

In [ ]:
loss, acc = model.evaluate(test_generator)
print(f"Test Accuracy: {acc:.4f}")

## 📈 Plot Accuracy

In [ ]:
plt.plot(history.history['accuracy'], label='Train Acc')
plt.plot(history.history['val_accuracy'], label='Val Acc')
plt.title('Training and Validation Accuracy')
plt.legend()
plt.savefig("accuracy_graph.png")
plt.show()

## 📋 Classification Report

In [ ]:
Y_pred = model.predict(test_generator)
y_pred = np.round(Y_pred).astype(int)
print('Confusion Matrix')
print(confusion_matrix(test_generator.classes, y_pred))
print('Classification Report')
print(classification_report(test_generator.classes, y_pred, target_names=['Normal', 'Pneumonia']))

## 💾 Save the Model

In [ ]:
model.save('pneumonia_detector_model.h5')